In [3]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from datetime import datetime, timedelta
from alpaca_trade_api import REST, TimeFrame
from arch import arch_model
import time

API_KEY = "PKL127PSJ903YEEWY3BW"
API_SECRET = "QvtKbQvYrHKKZkrTHoz3Shi3Org0pY5L0WtonI4i"
BASE_URL = "https://paper-api.alpaca.markets"

api = REST(API_KEY, API_SECRET, BASE_URL)
symbol = "AAPL"
print('Set up api')

end_date = datetime.today() - timedelta(days=1)
start_date = end_date - timedelta(days=300)
print('creating daily bars')

daily_bars = api.get_bars(
    symbol, TimeFrame.Day,
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d'), feed = 'iex'
).df

daily_df = daily_bars[['close']].copy()
daily_df.index = pd.to_datetime(daily_df.index)
daily_df['log_ret'] = np.log(daily_df['close']).diff()
daily_df['variance'] = daily_df['log_ret'].rolling(180).var()
daily_df = daily_df.dropna()

print('daily df created')

def predict_volatility(x):
    x_scaled = x * 100
    model = arch_model(x_scaled, p=1, q=3).fit(disp='off', update_freq=5)
    return model.forecast(horizon=1).variance.iloc[-1, 0] / 10000

daily_df['predictions'] = daily_df['log_ret'].rolling(180).apply(predict_volatility)
daily_df = daily_df.dropna()
daily_df['prediction_premium'] = (daily_df['predictions'] - daily_df['variance']) / daily_df['variance']
daily_df['premium_std'] = daily_df['prediction_premium'].rolling(180).std()

def get_daily_signal(row):
    if row['prediction_premium'] > row['premium_std']:
        return 1
    elif row['prediction_premium'] < -row['premium_std']*0.2:
        return -1
    return 0

daily_df['signal_daily'] = daily_df.apply(get_daily_signal, axis=1)
today = pd.Timestamp.today().normalize()
daily_bias = int(daily_df.loc[today, 'signal_daily']) if today in daily_df.index else 0
print(f"Today's bias: {daily_bias}")

intraday_data = pd.DataFrame(columns=["close"])
position = 0
trade_qty = 1

def generate_signal(latest_row, daily_bias, current_position):
    rsi = latest_row['rsi']
    close = latest_row['close']
    upper = latest_row['upper']
    lower = latest_row['lower']
    if daily_bias == 1 and rsi > 70 and close > upper and current_position <= 0:
        return 1
    if daily_bias == -1 and rsi < 50 and close < lower and current_position >= 0:
        return -1
    return 0

print("Polling minute bars… (Ctrl+C to stop)")
while True:
    try:
        end_inst_time = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
        start_inst_time = (datetime.utcnow() - timedelta(minutes=800)).replace(microsecond=0).isoformat() + "Z"
        print('getting bars for last 10 mins')
        bars = api.get_bars(
            symbol, TimeFrame.Minute,
            start=start_inst_time,
            end=end_inst_time,
            feed = 'iex'
        ).df
        print('checking if bar empty')
        if bars.empty:
            print("No bars returned (market closed?). Sleeping 60s.")
            time.sleep(60)
            continue

        new = bars[['close']].copy()
        new.index = pd.to_datetime(new.index)
        print('formatting intraday')
        intraday_data = new
        intraday_data = intraday_data[~intraday_data.index.duplicated(keep='last')].sort_index()
        print(intraday_data)

        print('Calculating parameters')
        intraday_data['rsi'] = ta.rsi(intraday_data['close'], length=14)
        print(intraday_data)
        bb = ta.bbands(intraday_data['close'], length=20)
        intraday_data['upper'] = bb['BBU_20_2.0']
        intraday_data['lower'] = bb['BBL_20_2.0']
        print(intraday_data)
        latest_row = intraday_data.iloc[-1]
        signal = generate_signal(latest_row, daily_bias, position)
        print('Signal work')
        if signal == 1:
            if position == -1:
                api.submit_order(symbol, qty=trade_qty, side='buy', type='market', time_in_force='day')
            position = 1
            print(f"Entered LONG at {latest_row['close']:.2f}")
        elif signal == -1:
            if position == 1:
                api.submit_order(symbol, qty=trade_qty, side='sell', type='market', time_in_force='day')
            position = -1
            print(f"Entered SHORT at {latest_row['close']:.2f}")
        else:
          print(f"No trade | close={latest_row['close']:.2f} rsi={latest_row['rsi']:.1f}")

        time.sleep(60)

    except KeyboardInterrupt:
        print("Stopping")
        break
    except Exception as e:
        print("Loop error:", e)
        time.sleep(10)


Set up api
creating daily bars
daily df created
Today's bias: 0
Polling minute bars… (Ctrl+C to stop)
getting bars for last 10 mins


C:\Users\CHANDRASHEKHAR BASU\AppData\Local\Temp\ipykernel_21712\430905658.py:75: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_inst_time = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
C:\Users\CHANDRASHEKHAR BASU\AppData\Local\Temp\ipykernel_21712\430905658.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_inst_time = (datetime.utcnow() - timedelta(minutes=800)).replace(microsecond=0).isoformat() + "Z"


checking if bar empty
formatting intraday
                             close
timestamp                         
2025-08-19 13:30:00+00:00  230.960
2025-08-19 13:31:00+00:00  231.240
2025-08-19 13:32:00+00:00  231.175
2025-08-19 13:33:00+00:00  231.180
2025-08-19 13:34:00+00:00  230.710
...                            ...
2025-08-19 19:25:00+00:00  230.000
2025-08-19 19:26:00+00:00  230.020
2025-08-19 19:27:00+00:00  230.090
2025-08-19 19:28:00+00:00  230.110
2025-08-19 19:29:00+00:00  230.135

[356 rows x 1 columns]
Calculating parameters
                             close        rsi
timestamp                                    
2025-08-19 13:30:00+00:00  230.960        NaN
2025-08-19 13:31:00+00:00  231.240        NaN
2025-08-19 13:32:00+00:00  231.175        NaN
2025-08-19 13:33:00+00:00  231.180        NaN
2025-08-19 13:34:00+00:00  230.710        NaN
...                            ...        ...
2025-08-19 19:25:00+00:00  230.000  59.457288
2025-08-19 19:26:00+00:00  230.020  60.293

C:\Users\CHANDRASHEKHAR BASU\AppData\Local\Temp\ipykernel_21712\430905658.py:75: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_inst_time = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
C:\Users\CHANDRASHEKHAR BASU\AppData\Local\Temp\ipykernel_21712\430905658.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_inst_time = (datetime.utcnow() - timedelta(minutes=800)).replace(microsecond=0).isoformat() + "Z"


getting bars for last 10 mins
checking if bar empty
formatting intraday
                             close
timestamp                         
2025-08-19 13:30:00+00:00  230.960
2025-08-19 13:31:00+00:00  231.240
2025-08-19 13:32:00+00:00  231.175
2025-08-19 13:33:00+00:00  231.180
2025-08-19 13:34:00+00:00  230.710
...                            ...
2025-08-19 19:25:00+00:00  230.000
2025-08-19 19:26:00+00:00  230.020
2025-08-19 19:27:00+00:00  230.090
2025-08-19 19:28:00+00:00  230.110
2025-08-19 19:29:00+00:00  230.135

[356 rows x 1 columns]
Calculating parameters
                             close        rsi
timestamp                                    
2025-08-19 13:30:00+00:00  230.960        NaN
2025-08-19 13:31:00+00:00  231.240        NaN
2025-08-19 13:32:00+00:00  231.175        NaN
2025-08-19 13:33:00+00:00  231.180        NaN
2025-08-19 13:34:00+00:00  230.710        NaN
...                            ...        ...
2025-08-19 19:25:00+00:00  230.000  59.457288
2025-08-19 1